In [1]:
import json
import random

In [9]:
DATA_PATH = '../data/synthetic_pii_ner.json'
MODEL_PATH = '../models/gliner_medium-v2.1/'

In [8]:
! wget https://huggingface.co/datasets/urchade/synthetic-pii-ner-mistral-v1/resolve/main/data.json

In [10]:
with open(DATA_PATH, "r") as f:
    data = json.load(f)

print('Dataset size:', len(data))

random.shuffle(data)
print('Dataset is shuffled...')

train_dataset = data[:int(len(data)*0.9)]
test_dataset = data[int(len(data)*0.9):]

print('Dataset is splitted...')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
import pandas as pd
train_df = pd.DataFrame(train_dataset)
train_df.head(5)

,tokenized_text,ner
0,"[Juan, Pérez, ,, un, jugador, habitual, del, C...","[[0, 1, person], [7, 10, organization], [23, 2..."
1,"[Mme, Yoshiko, Nakamura, ,, agricultrice, de, ...","[[0, 2, person], [43, 45, address], [38, 41, s..."
2,"[Prof, ., Dr, ., Abeba, Tesfamariam, ,, gebore...","[[0, 5, person], [25, 25, degree], [58, 58, de..."
3,"[Hier, ist, das, Auszug, aus, dem, Protokoll, ...","[[24, 25, person], [32, 32, membership number]..."
4,"[El, Sr, ., Juan, Pérez, ,, un, empresario, de...","[[3, 4, person], [18, 19, organization], [23, ..."


In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import torch
from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset

/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

model = GLiNER.from_pretrained(MODEL_PATH)

config.json not found in /workspaces/NER-project/models/gliner_medium-v2.1
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# use it for better performance, it mimics original implementation but it's less memory efficient
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

In [8]:
model.to(device)
print("done")

done


In [16]:
# calculate number of epochs
num_steps = 500
batch_size = 8
data_size = len(train_dataset)
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

training_args = TrainingArguments(
    output_dir=MODEL_PATH,
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    save_steps = 100,
    save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
)

trainer.train()

/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Step,Training Loss,Validation Loss
500,93.875500,68.641869
1000,65.905000,61.758347
1500,60.236400,57.332455
2000,59.639900,56.785431


/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/gliner/data_processing/processor.py:206: UserWarning: Sentence of length 503 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/gliner/data_processing/processor.py:206: UserWarning: Sentence of length 503 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/gliner/data_processing/processor.py:206: UserWarning: Sentence of length 404 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/opt/conda/envs/training-pipeline/lib/python3.10/site-packages/gliner/data_processing/processor.py:206: UserWarning: Sentence of length 503 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len

TrainOutput(global_step=2209, training_loss=68.96932072982685, metrics={'train_runtime': 1478.6641, 'train_samples_per_second': 11.951, 'train_steps_per_second': 1.494, 'total_flos': 0.0, 'train_loss': 68.96932072982685, 'epoch': 1.0})

In [11]:
import pandas as pd

train_df = pd.DataFrame(train_dataset)

In [21]:
text = """
Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards,[note 3] a record three UEFA Men's Player of the Year Awards, and four European Golden Shoes, the most by a European player. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (128) and international appearances (205). He is one of the few players to have made over 1,200 professional career appearances, the most by an outfield player, and has scored over 850 official senior career goals for club and country, making him the top goalscorer of all time.
"""

# Labels for entity prediction
labels = ["Person", "Award"] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

Cristiano Ronaldo dos Santos Aveiro => Person
Ballon d'Or => Award
UEFA Men's Player of the Year Awards => Award
European Golden Shoes => Award
